## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import optuna

# from lightgbm import LGBMRegressor
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

## Load Dataset

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
categorical_cols = ['cat'+str(i) for i in range(10)]
continuous_cols = ['cont'+str(i) for i in range(14)]

## Encode categorical features

In [ ]:
for e in categorical_cols:
    le = LabelEncoder()
    train[e] = le.fit_transform(train[e])
    test[e] = le.transform(test[e])

In [ ]:
train.head()

In [ ]:
data = train[categorical_cols+continuous_cols]
target = train['target']

In [ ]:
data

In [ ]:
target

## Optimization function using Optuna

In [ ]:
def objective(trial,data=data,target=target):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2, random_state=777)
    
    param = {
        'tree_method': 'gpu_hist',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree' : trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample' : trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'learning_rate' : trial.suggest_categorical('learning_rate', [0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02]),
        'n_estimators' : 10000,
        'max_depth' : trial.suggest_categorical('max_depth', [5, 7, 9, 11, 13, 15, 17]),
        'random_state' : trial.suggest_categorical('random_state', [777]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'early_stopping_rounds': 100,
    }
    model = xgb.XGBRegressor(**param)
    
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)], verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds, squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.trials_dataframe()

## Optuna Quick Visualization 

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_edf(study)

## create XGBoostRegressor model with the best hyperparameters

In [ ]:
best_trial = study.best_trial.params
best_trial['n_estimators'] = 10000
best_trial['tree_method'] = 'gpu_hist'
best_trial['early_stopping_rounds'] = 100
best_trial

In [ ]:
columns = categorical_cols + continuous_cols
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5, random_state=777, shuffle=True)
rmse = []
n = 0
for trn_idx, test_idx in kf.split(train[columns],train['target']):
    X_tr, X_val = train[columns].iloc[trn_idx], train[columns].iloc[test_idx]
    y_tr, y_val = train['target'].iloc[trn_idx], train['target'].iloc[test_idx]
    model = xgb.XGBRegressor(**best_trial)
    model.fit(X_tr, y_tr, eval_set=[(X_val,y_val)], verbose=False)
    preds += model.predict(test[columns]) / kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(f'fold: {n+1} ==> rmse: {rmse[n]}')
    n += 1

In [ ]:
np.mean(rmse)

In [ ]:
preds

In [ ]:
len(preds)

In [ ]:
sub['target'] = preds
sub.to_csv('submission.csv', index=False)